In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-opt")


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-187158
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-187158


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
cluster_name = "criscluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import shutil
from scipy.stats import loguniform
from azureml.train.hyperdrive import BanditPolicy

# Parameter sampler suitable for logistic regression
ps = RandomParameterSampling(
    {
#        '--C' : loguniform(1e-5, 100),
        '--C': choice(0.1,1.0,10),
        '--max_iter': choice(100,200,300)
    }
)


# Specify a Policys
policy = BanditPolicy(slack_factor = 0.1,
            evaluation_interval = 1,
            delay_evaluation = 5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

script_folder = "./training"    
    
# Copy the train.py into the script folder
shutil.copy('train.py', script_folder)

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./training',
        script = 'train.py',
        command=['ls', '-l'],
        compute_target=cluster_name,
        environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config = src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=6,
                                    max_concurrent_runs=4)


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config = hyperdrive_config, show_output = True)

TrainingException: TrainingException:
	Message: Exception occurred while creating the HyperDrive run: [Operation returned an invalid status code 'BAD REQUEST']. Please try again later.
	InnerException Operation returned an invalid status code 'BAD REQUEST'
	ErrorResponse 
{
    "error": {
        "code": "SystemError",
        "message": "Exception occurred while creating the HyperDrive run: [Operation returned an invalid status code 'BAD REQUEST']. Please try again later.",
        "inner_error": {
            "code": "ClientError",
            "inner_error": {
                "code": "HyperDriveTrainingError"
            }
        }
    }
}

In [ ]:
hyperdrive_run.get_status()
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
best_run, best_model = run.get_output()
best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###